# Loading Libraries

In [2]:
import pandas as pd
import plotly.express as px
import os
import statsmodels
from datetime import date
import copy
os.chdir('/home/code/')

# Loading dfs

In [3]:
sales = pd.read_parquet('sales.parquet')
customers = pd.read_parquet('customers.parquet')
items = pd.read_parquet('items.parquet')
cities = pd.read_parquet('cities.parquet')


# Exploring dfs

In [4]:
sales.head(4)

,createdBy,customerID,ticketNumberDetail,value,idType,date,ticketNumber,paymentMethod,itemID,discount,storeID,salesID
0,None,None,None,NaN,None,2021-06-11,183C93C6-08CB-EB11-B636-AC82478FB6A9,None,NaN,NaN,10,183C93C6-08CB-EB11-B636-AC82478FB6A9
1,None,None,None,NaN,None,2021-06-11,A938A208-09CB-EB11-B638-AC82478FA8AD,None,NaN,NaN,10,A938A208-09CB-EB11-B638-AC82478FA8AD
2,victor,26557317,7AC93ADA-B4CA-EB11-B636-AC82478FB6A9,51100.0,C,2021-06-11,916A5D76-B4CA-EB11-B636-AC82478FB6A9,CON,746.0,0.0,10,916A5D76-B4CA-EB11-B636-AC82478FB6A97AC93ADA-B...
3,victor,26557317,D44DFE12-B5CA-EB11-B636-AC82478FB6A9,1.0,C,2021-06-11,916A5D76-B4CA-EB11-B636-AC82478FB6A9,CON,7530.0,0.0,10,916A5D76-B4CA-EB11-B636-AC82478FB6A9D44DFE12-B...


In [5]:
customers.head(4)

,custCreationDate,cityId,custHood,birthDate,customerID,custEmail,gender,status
index,,,,,,,,
2,2013-01-26 00:00:00.000,1.0,None,2013-01-26 00:00:00.000,26557317,None,0.0,1
4,2021-03-18 00:00:00.000,1.0,None,1992-12-15 00:00:00.000,2041175771,SINOYKARI@GMAIL.COM,1.0,1
7,2021-05-19 00:00:00.000,1.0,None,None,22576702,None,0.0,1
8,2013-02-13 00:00:00.000,1.0,None,None,691301004,secostasas@gmail.com,0.0,1


In [6]:
items.head(4)

,itemName,itemApplications,brand,itemLine,itemRegulated,businessUnit,unitOfMeasure,itemCreationDate,itemGroup,itemSubline,category,itemID,itemLevel
index,,,,,,,,,,,,,
2,KYROFOSFAN NF FCO X 100 ML,GANADERÍA BOVINA,KYROVET,VITAMINAS Y MINERALES,NO,AGROPECUARIO,FCO,2012-11-21 11:37:11.000,FARMACEUTICOS,FOSFORO PREPARACIONES,None,746.0,GAMA BAJA
3,IMPOCONSUMO BOLSA PLASTICA,CONSUMO,VARIOS,IMPLEMENTOS VARIOS,NO,CONSUMO,UND,2017-07-12 18:03:36.000,LINEA VARIOS,None,None,7530.0,GAMA BAJA
4,PEDIGREE MINI BISCUIT X 100 GRMS,MASCOTAS,PEDIGREE,ALIMENTO PARA PERROS,NO,MASCOTAS,UND,2019-10-02 14:59:21.000,ALIMENTOS CONCENTRADOS,SNACKS,None,8657.0,GAMA MEDIA
6,AGILITY PEQUEÑOS CACHORROS X 3 KG,MASCOTAS,ITALCOL MASCOTAS,ALIMENTO PARA PERROS,NO,MASCOTAS,BOL,2016-08-24 11:42:55.000,ALIMENTOS CONCENTRADOS,SEGMENTO HIGH PREMIUM,PERROS,6791.0,GAMA ALTA


In [11]:
cities.head(4)

,cityId,cityGrouper
index,,
2,1.0,Capital
149,770.0,Non Capital
246,372.0,Non Capital
304,836.0,Non Capital


# Business Problems

1. How are our sales grow by day, month and year? Is there a seasonal trend?

In [12]:
sales_ex_1 = sales[['value', 'date']]

In [15]:
sales_ex_1_day = sales_ex_1.groupby(by='date').sum().reset_index().sort_values(by='date')
sales_ex_1_month = copy.deepcopy(sales_ex_1)
sales_ex_1_month['month_year'] = pd.to_datetime(sales_ex_1.date).dt.to_period('M')
sales_ex_1_month = sales_ex_1_month.groupby(by='month_year').sum().reset_index().sort_values(by='month_year')
sales_ex_1_month.month_year = sales_ex_1_month.month_year.astype('str')
sales_ex_1_year = copy.deepcopy(sales_ex_1)
sales_ex_1_year['year'] = pd.to_datetime(sales_ex_1.date).dt.year
sales_ex_1_year = sales_ex_1_year.groupby(by='year').sum().reset_index().sort_values(by='year')
sales_ex_1_year.year = sales_ex_1_year.year.astype('str')

In [16]:
fig = px.line(sales_ex_1_day, x="date", y="value", title='Sales by day')
fig.update_traces(line_color='#1C70C9')
fig.show()

In [17]:
fig = px.line(sales_ex_1_month, x="month_year", y="value", title='Sales by month')
fig.update_traces(line_color='#A31F34')
fig.show()

In [18]:
fig = px.line(sales_ex_1_year, x="year", y="value", title='Sales by year')
fig.show()

There is a problem with the last two graphs. We don have full info about October and the 2021 for make a good comparison. In the case of the month periodical graph the solution could be skip October-2021 and take into account the other months. In case of year graph is not as simple because we only have two years, then could be better to use the mean instead of the sum of values by day. 

In [19]:
# Months without October-2021
fig = px.line(sales_ex_1_month.iloc[:-1, :], x="month_year", y="value", title='Sales by month')
fig.update_traces(line_color='#A31F34')
fig.show()

In [20]:
# average_year
sales_ex_1_year = copy.deepcopy(sales_ex_1)
sales_ex_1_year['year'] = pd.to_datetime(sales_ex_1.date).dt.year
sales_ex_1_year = sales_ex_1_year.groupby(by='year').mean().reset_index().sort_values(by='year')
sales_ex_1_year.year = sales_ex_1_year.year.astype('str')
fig = px.line(sales_ex_1_year, x="year", y="value", title='Sales by year')
fig.show()

That shows the opposite trend! Which means that the average sales in 2021 have been better than the average sales in 2021

Is there an seasonal trend? We can't say that watching year and month plots but lets take a closer look in days. What if we analyze sales by day of week? What could we find?

In [21]:
sales_ex_1_day = sales_ex_1.groupby(by='date').sum().reset_index().sort_values(by='date')
sales_ex_1_day['day_of_week'] = pd.to_datetime(sales_ex_1_day.date).dt.weekday
sales_ex_1_day = sales_ex_1_day.groupby(by='day_of_week').mean().reset_index()
replace_days = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
sales_ex_1_day['day_name'] = sales_ex_1_day['day_of_week'].replace(replace_days)

In [22]:
fig = px.bar(sales_ex_1_day, x='day_name', y='value', color='day_name', title='Average of sales by weekday')
fig.show()

From the graph above we can conclude that there is a clear seasonal trend: the Sundays the average value of sales decreases a lot, maybe because that day the store is not open all the day or is just working online.

In [23]:
# Cleaning memory

del replace_days
del fig
del sales_ex_1
del sales_ex_1_day 
del sales_ex_1_month 
del sales_ex_1_year

------------------------

2. How is the distribution of our sales by the use/application of each item for our customer

First, we see that there is a lot more productos for pets than for other categories. 

In [24]:
item_applications_ex_2 = items.itemApplications.value_counts().reset_index().rename(columns = {
                                                            'index':'itemApplications', 
                                                            'itemApplications':'quantity'})

In [57]:
fig = px.pie(item_applications_ex_2, values='quantity', names='itemApplications', 
             title='Quantity of items by itemApplications')
fig.show()

Does that trend remains over our sales?

In [50]:
sales_ex_2 = copy.deepcopy(sales)
items_ex_2 = copy.deepcopy(items)
sales_ex_2 = sales_ex_2[['itemID', 'value']]
sales_ex_2 = sales_ex_2.dropna()
items_ex_2 = items_ex_2[['itemID', 'itemApplications']]
merge_ex_2 = sales_ex_2.merge(items_ex_2, how='inner', left_on='itemID', right_on='itemID').drop(columns='itemID')

In [53]:
merge_ex_2_sum = merge_ex_2.groupby(by='itemApplications').sum().reset_index()
fig = px.pie(merge_ex_2_sum, values='value', names='itemApplications', title='Sales by itemApplications')
fig.show()

Actually the sales of each category has correspondence with the quantity of products that we have per each line. This can indicates a behaviour that the shop has been having across its existence: The product lines with more sales have been increasing faster than the others. However, the participation of pet products on the total sales is smaller than its participation over items and the participation of agricultural products in sales is much bigger than its participation on items. Maybe could be interesting to explore the possibility of increase the item stock that we have on agricultural products in the future. Lets see another chart of average sale by each category. take into account that this average is done over the Ticket detailed and not over the whole sale done by a client at a time.

In [56]:
merge_ex_2_mean = merge_ex_2.groupby(by='itemApplications').mean().reset_index()
fig = px.pie(merge_ex_2_mean, values='value', names='itemApplications', title='Average sales by itemApplications')
fig.show()

What a surprise! Products with small participation over the item type and the total sales have the biggest average sale value! What can that mean? Maybe those are the most expensive products but just a little portion of customers buy into this line. Thats an attention call, there could be a big market concentration in this products.

In [58]:
# Cleaning memory

del item_applications_ex_2
del sales_ex_2 
del items_ex_2
del merge_ex_2
del merge_ex_2_sum
del merge_ex_2_mean
del fig

--------------------------

3. Why our sales have fall from Jun 2021 to Sept 2021?

In [190]:
grouper_list = ['itemApplications', 'itemRegulated',
       'businessUnit', 'itemGroup', 'category', 'itemLevel']
for grouper_variable in grouper_list:
    sales_ex_3 = copy.deepcopy(sales)
    items_ex_3 = copy.deepcopy(items)
    sales_ex_3 = sales_ex_3[['date','itemID', 'value']]
    sales_ex_3 = sales_ex_3.dropna()
    items_ex_3 = items_ex_3[['itemID', grouper_variable]]
    merge_ex_3 = sales_ex_3.merge(items_ex_3, how='inner', left_on='itemID', right_on='itemID').drop(columns='itemID')
    merge_ex_3.date = pd.to_datetime(merge_ex_3.date)
    merge_ex_3.date = merge_ex_3.date.dt.to_period('M')
    merge_ex_3 = merge_ex_3.sort_values(by='date').groupby(by=['date', grouper_variable]).sum().reset_index()
    merge_ex_3.date = merge_ex_3.date.astype(str)
    fig = px.area(merge_ex_3, x="date", y="value", color=grouper_variable, title=f'Area sales by {grouper_variable}')
    fig.show()

In [203]:
grouper_list = ['cityId', 'gender', 'PDV_or_not']
for grouper_variable in grouper_list:
    sales_ex_3 = copy.deepcopy(sales)
    items_ex_3 = copy.deepcopy(customers)
    sales_ex_3 = sales_ex_3[['date','customerID', 'value']]
    sales_ex_3 = sales_ex_3.dropna()
    items_ex_3 = items_ex_3[['customerID', grouper_variable]]
    merge_ex_3 = sales_ex_3.merge(items_ex_3, how='inner', left_on='customerID', 
                                  right_on='customerID').drop(columns='customerID')
    merge_ex_3.date = pd.to_datetime(merge_ex_3.date)
    merge_ex_3.date = merge_ex_3.date.dt.to_period('M')
    merge_ex_3 = merge_ex_3.sort_values(by='date').groupby(by=['date', grouper_variable]).sum().reset_index()
    merge_ex_3.date = merge_ex_3.date.astype(str)
    fig = px.area(merge_ex_3, x="date", y="value", color=grouper_variable, title=f'Area sales by {grouper_variable}')
    fig.show()

In [205]:
grouper_list = ['createdBy', 'idType','paymentMethod','storeID']
for grouper_variable in grouper_list:
    sales_ex_3 = copy.deepcopy(sales)
    items_ex_3 = copy.deepcopy(customers)
    sales_ex_3 = sales_ex_3[['date','customerID', 'value', grouper_variable]]
    sales_ex_3 = sales_ex_3.dropna()
    items_ex_3 = items_ex_3[['customerID']]
    merge_ex_3 = sales_ex_3.merge(items_ex_3, how='inner', left_on='customerID', 
                                  right_on='customerID').drop(columns='customerID')
    merge_ex_3.date = pd.to_datetime(merge_ex_3.date)
    merge_ex_3.date = merge_ex_3.date.dt.to_period('M')
    merge_ex_3 = merge_ex_3.sort_values(by='date').groupby(by=['date', grouper_variable]).sum().reset_index()
    merge_ex_3.date = merge_ex_3.date.astype(str)
    fig = px.area(merge_ex_3, x="date", y="value", color=grouper_variable, title=f'Area sales by {grouper_variable}')
    fig.show()

All the variables seem to fall uniformly, so the drop in sales does not seem to be associated with the drop in any special category of sales. It seems that the last few months have been randomly chosen, gradually reducing the amount of daily sales or their value. 